In [ ]:
from owslib.wms import WebMapService #required for connecting to WMS servers
from matplotlib import pyplot as plt #required for creating the graph
import imageio #reads and saves many types of images files
import shapefile #reads shapefiles
import os #only to delete temporal files (uncomment below)

def wms_download(id_name,path,x,y,server,layer):
    tam=50
    wms = WebMapService(server)
    img_area=wms.getmap(layers=[layer],srs='EPSG:25829',bbox=(x-tam,y-tam,x+tam,y+tam),format='image/jpeg',size=(400,400))
    out = open(path+id_name+'_'+layer+'.jpg', 'wb')
    out.write(img_area.read())
    out.close()

path='/home/user1/example2/'

wmsurl=[]
wmslayers=[]
wmsfile = open(path+'wms_servers.txt')
for line in wmsfile:
    wmsurl.append(line.split(',')[0])
    wmslayers.append((line.split(',')[1]).split('\n')[0])


sf = shapefile.Reader(path+'sites.shp')
shapes = sf.shapes()
sf.fields
sr = sf.shapeRecords()

pos=0
for pts in shapes:
    name=sr[pos].record[1]
    pos=pos+1
    x,y=pts.points[0][0],pts.points[0][1]
    plt.figure()
    plot_pos=200+(((len(wmslayers)/2)+(len(wmslayers)/2)%2)*10)+1
    for server,layer in zip(wmsurl,wmslayers):
        wms_download(name,path,x,y,server,layer)
        plt.figure(1,figsize=(14,11))
        plt.subplot(plot_pos)
        plt.axis('off')
        plt.title(layer)
        image=imageio.imread(path+name+'_'+layer+'.jpg')
        plt.imshow(image)
        #os.system('rm '+path+name+'_'+layer+'.jpg')
        plot_pos=plot_pos+1
    plt.savefig(path+name+'.pdf')